In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [22]:
# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [33]:
# Hyper parameters
num_epochs = 100
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [14]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

In [15]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [29]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,16,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size=5,stride=1,padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        self.fc = nn.Linear(7*7*32,num_classes)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0),-1)
        out = self.fc(out)
        return out
    
model = ConvNet(num_classes).to(device)

In [26]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [35]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs) :
    for i,(images,labels) in enumerate(train_loader) :
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward
        outputs = model(images)
        loss = criterion(outputs,labels)
        
        #backward pass and optim
        optimizer.zero_grad()
        loss.backward
        optimizer.step()
        if (i+1) % 600 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/100], Step [600/600], Loss: 2.3355
Epoch [2/100], Step [600/600], Loss: 2.2989
Epoch [3/100], Step [600/600], Loss: 2.3633
Epoch [4/100], Step [600/600], Loss: 2.3624
Epoch [5/100], Step [600/600], Loss: 2.3196
Epoch [6/100], Step [600/600], Loss: 2.2841
Epoch [7/100], Step [600/600], Loss: 2.3303
Epoch [8/100], Step [600/600], Loss: 2.3570
Epoch [9/100], Step [600/600], Loss: 2.3503
Epoch [10/100], Step [600/600], Loss: 2.3234
Epoch [11/100], Step [600/600], Loss: 2.2816
Epoch [12/100], Step [600/600], Loss: 2.3147
Epoch [13/100], Step [600/600], Loss: 2.3602
Epoch [14/100], Step [600/600], Loss: 2.3529
Epoch [15/100], Step [600/600], Loss: 2.3546
Epoch [16/100], Step [600/600], Loss: 2.3795
Epoch [17/100], Step [600/600], Loss: 2.3859
Epoch [18/100], Step [600/600], Loss: 2.2948
Epoch [19/100], Step [600/600], Loss: 2.3520
Epoch [20/100], Step [600/600], Loss: 2.3248
Epoch [21/100], Step [600/600], Loss: 2.3438
Epoch [22/100], Step [600/600], Loss: 2.3274
Epoch [23/100], Ste

In [36]:
# Test the model
model.eval() # eval mode (batchnorm uses moving mean/variance instead of mini-batch
with torch.no_grad() :
    correct = 0
    total = 0 
    for images,labels in test_loader :
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 12.82 %


In [31]:
# Save the model checkpoint
torch.save(model.state_dict(), 'CNN_model.ckpt')